### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [ ]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

file = 'test_raw_16k.h5py'
H5_dic = read_data_header()
chunk_size = 512*10

males = H5PY_RW().open_h5_dataset(file, subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset(file, subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
N = 256
max_pool = 128
batch_size = 8
learning_rate = 0.001

config_model = {}
config_model["type"] = "pretraining"

config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size

config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["window"] = 1024

config_model["smooth_size"] = 20

config_model["alpha"] = learning_rate
config_model["reg"] = 1e-4
config_model["beta"] = 0.1
config_model["rho"] = 0.01

idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
batch_size = 1
config_model["batch_size"] = batch_size
config_model["type"] = "DAS_train_front"

In [5]:
from models.adapt import Adapt
import config

full_id = 'soft-base-9900'+idd

folder='DAS_train_front'
model = Adapt(config_model=config_model,pretraining=False)
model.create_saver()

path = os.path.join(config.workdir, 'floydhub_model', "pretraining")
# path = os.path.join(config.log_dir, "pretraining")
model.restore_model(path, full_id)

ID : AdaptiveNet-orange-fire-5150
INFO:tensorflow:Restoring parameters from /home/anthony/das/floydhub_model/pretraining/AdaptiveNet-soft-base-9900-N=256--alpha=0.001--batch_size=8--beta=0.1--chunk_size=5120--maxpool=128--reg=0.0001--rho=0.01--smooth_size=20--type=pretraining--window=1024-/model.ckpt


## Connect DAS model to the front end

In [6]:
from models.das import DAS

with model.graph.as_default():
    model.connect_front(DAS)
    model.sepNet.output = model.sepNet.prediction
    model.cost = model.sepNet.cost
    model.freeze_front()
    model.optimize
    model.tensorboard_init()

<utils.ops.ops.Conv2D instance at 0x7f3bac0cd710>
<utils.ops.ops.BatchNorm instance at 0x7f3bac0cd5f0>
<utils.ops.ops.Activation instance at 0x7f3bac0cd950>
<utils.ops.ops.Dropout instance at 0x7f3bac0cd7e8>
<utils.ops.ops.Conv2D instance at 0x7f3bac0cd7a0>
<utils.ops.ops.BatchNorm instance at 0x7f3bac0cd758>
<utils.ops.ops.Activation instance at 0x7f3bac0cd998>
<utils.ops.ops.Conv2D instance at 0x7f3bac0cd9e0>
<utils.ops.ops.BatchNorm instance at 0x7f3bac0cda28>
<utils.ops.ops.Activation instance at 0x7f3bac0cda70>
<utils.ops.ops.Dropout instance at 0x7f3bac0cdab8>
<utils.ops.ops.Conv2D instance at 0x7f3bac0cdb00>
<utils.ops.ops.BatchNorm instance at 0x7f3bac0cdb48>
<utils.ops.ops.Activation instance at 0x7f3bac0cdb90>
<utils.ops.ops.Conv2D instance at 0x7f3bac0cdbd8>
<utils.ops.ops.BatchNorm instance at 0x7f3bac0cdc20>
<utils.ops.ops.Activation instance at 0x7f3bac0cdc68>
<utils.ops.ops.Dropout instance at 0x7f3bac0cdcb0>
<utils.ops.ops.Conv2D instance at 0x7f3bac0cdcf8>
<utils.ops.o

In [7]:
from itertools import compress
with model.graph.as_default():
    global_vars = tf.global_variables()
    is_not_initialized = model.sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))
    print not_initialized_vars
    if len(not_initialized_vars):
        init = tf.variables_initializer(not_initialized_vars)


[<tf.Variable 'centroids:0' shape=(40, 40) dtype=float32_ref>, <tf.Variable 'prediction/W:0' shape=(128, 40) dtype=float32_ref>, <tf.Variable 'prediction/Variable:0' shape=(40,) dtype=float32_ref>, <tf.Variable 'prediction/conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>, <tf.Variable 'prediction/conv2d/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/gamma:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/moving_mean:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization/moving_variance:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/conv2d_1/kernel:0' shape=(3, 3, 32, 32) dtype=float32_ref>, <tf.Variable 'prediction/conv2d_1/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalization_1/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'prediction/batch_normalizatio

# Model creation

# Pretraining the model 

In [8]:
nb_iterations = 1000

#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in,learning_rate, i, ind_train=Ind)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)

Step # 0  loss= 0.693147
DAS model saved at iteration number  0  with cost =  0.693147
Step # 1  loss= 0.693172
Step # 2  loss= 0.692207
Step # 3  loss= 0.691733
Step # 4  loss= 0.69277
Step # 5  loss= 0.695828
Step # 6  loss= 0.694639
Step # 7  loss= 0.69309
Step # 8  loss= 0.690242
Step # 9  loss= 0.692835
Step # 10  loss= 0.69209
Step # 11  loss= 0.695121
Step # 12  loss= 0.687855
Step # 13  loss= 0.691004
Step # 14  loss= 0.693573
Step # 15  loss= 0.699587
Step # 16  loss= 0.692351
Step # 17  loss= 0.69584
Step # 18  loss= 0.694558


KeyboardInterrupt: 